# Chapter 15: Self-Attention and Transformers
## The Attention mechanism and its implementation in PyTorch
Computing self-attention of a sentence with GloVe embeddings and the `MultiheadAttention` class with PyTorch

Programs from the book: [_Python for Natural Language Processing_](https://link.springer.com/book/9783031575488)

__Author__: Pierre Nugues

## Modules

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
torch.manual_seed(1234)

## Noncontextual embeddings

We load GloVe

In [3]:
def read_embeddings(file):
    """
    Return the embeddings in the from of a dictionary
    :param file:
    :return:
    """
    embeddings = {}
    with open(file, encoding='utf8') as glove:
        for line in glove:
            values = line.strip().split()
            word = values[0]
            vector = [float(value) for value in values[1:]]
            vector = torch.FloatTensor(vector)
            embeddings[word] = vector
    return embeddings

In [4]:
PATH = '../../corpus/'

In [5]:
embedding_file = PATH + 'glove.6B.50d.txt'
embeddings_dict = read_embeddings(embedding_file)

In [6]:
embeddings_dict['ship']

tensor([ 1.5213,  0.1052,  0.3816, -0.5080,  0.0324, -0.1348, -1.2474,  0.7981,
         0.8469, -1.1010,  0.8874,  1.3749,  0.4293,  0.6572, -0.2636, -0.4176,
        -0.4885,  0.9106, -1.7158, -0.4380,  0.7839,  0.1964, -0.4066, -0.5397,
         0.8244, -1.7434,  0.1428,  0.2804,  1.1688,  0.1690,  2.2271, -0.5827,
        -0.4572,  0.6281,  0.5444,  0.2846,  0.4448, -0.5534, -0.3649, -0.0164,
         0.4088, -0.8715,  1.5513, -0.8070, -0.1004, -0.2846, -0.3322, -0.5061,
         0.4827, -0.6620])

## Cosine similarity

Let us compute the cosine similarity of the words in a sentence:
> I must go back to my ship and to my crew

_Odyssey_, book I 

Remember that:
$$\cos(\mathbf{u}, \mathbf{v}) = \frac{\mathbf{u} \cdot \mathbf{v}}{||\mathbf{u}|| \cdot ||\mathbf{v} ||}$$

In [7]:
sentence_odyssey = 'I must go back to my ship and to my crew'
sentence_amazon = 'We process and ship your order'
# in the most cost-efficient way possible

In [8]:
words_a = sentence_amazon.lower().split()
words_o = sentence_odyssey.lower().split()
words_o

['i', 'must', 'go', 'back', 'to', 'my', 'ship', 'and', 'to', 'my', 'crew']

We build the embedding matrix

In [9]:
def embedding_matrix(words, embeddings_dict):
    embeddings_seq = [embeddings_dict[word] for word in words]
    embeddings_seq = torch.stack(embeddings_seq)
    return embeddings_seq

In [10]:
X_a = embedding_matrix(words_a, embeddings_dict)
X_o = embedding_matrix(words_o, embeddings_dict)

In [11]:
X_o.size()

torch.Size([11, 50])

In [12]:
X_a.size()

torch.Size([6, 50])

In [13]:
X_o[0][:10]

tensor([ 1.1891e-01,  1.5255e-01, -8.2073e-02, -7.4144e-01,  7.5917e-01,
        -4.8328e-01, -3.1009e-01,  5.1476e-01, -9.8708e-01,  6.1757e-04])

We compute the attention weights as the pairwise cosines of the word embeddings

In [14]:
def attn_cos_weights(embeddings_mat):
    E_normed = F.normalize(embeddings_mat)
    attn_weights_cos = E_normed @ E_normed.T
    return attn_weights_cos

In [15]:
def print_cos_weights(words, embeddings_dict):
    embeddings = embedding_matrix(words, embeddings_dict)
    attn_weights_cos = attn_cos_weights(embeddings)
    print('\t', end='')
    for i in range(len(words)):
        print(words[i], end='\t')
    print()

    for i in range(attn_weights_cos.shape[0]):
        print(words[i], end='\t')
        for j in range(attn_weights_cos.shape[1]):
            print(f"{attn_weights_cos[i,j]:.2f}", end='\t')
        print()

In [16]:
print_cos_weights(words_o, embeddings_dict)

	i	must	go	back	to	my	ship	and	to	my	crew	
i	1.00	0.75	0.86	0.76	0.73	0.90	0.35	0.65	0.73	0.90	0.42	
must	0.75	1.00	0.85	0.68	0.87	0.69	0.42	0.69	0.87	0.69	0.45	
go	0.86	0.85	1.00	0.84	0.84	0.81	0.41	0.68	0.84	0.81	0.49	
back	0.76	0.68	0.84	1.00	0.83	0.76	0.49	0.77	0.83	0.76	0.51	
to	0.73	0.87	0.84	0.83	1.00	0.68	0.54	0.86	1.00	0.68	0.51	
my	0.90	0.69	0.81	0.76	0.68	1.00	0.38	0.63	0.68	1.00	0.44	
ship	0.35	0.42	0.41	0.49	0.54	0.38	1.00	0.46	0.54	0.38	0.78	
and	0.65	0.69	0.68	0.77	0.86	0.63	0.46	1.00	0.86	0.63	0.49	
to	0.73	0.87	0.84	0.83	1.00	0.68	0.54	0.86	1.00	0.68	0.51	
my	0.90	0.69	0.81	0.76	0.68	1.00	0.38	0.63	0.68	1.00	0.44	
crew	0.42	0.45	0.49	0.51	0.51	0.44	0.78	0.49	0.51	0.44	1.00	


In [17]:
print_cos_weights(words_a, embeddings_dict)

	we	process	and	ship	your	order	
we	1.00	0.64	0.70	0.36	0.75	0.64	
process	0.64	1.00	0.61	0.29	0.52	0.67	
and	0.70	0.61	1.00	0.46	0.58	0.69	
ship	0.36	0.29	0.46	1.00	0.37	0.52	
your	0.75	0.52	0.58	0.37	1.00	0.63	
order	0.64	0.67	0.69	0.52	0.63	1.00	


## Contextual embeddings

We design a new vector representation for _ship_ so that it receives an influence from _crew_ and the other words of its context. This influence will depend on the embeddings from te context. Let us use the cosine similarities as attention weights

In [18]:
attn_cos_weights(X_o)[6]

tensor([0.3466, 0.4178, 0.4068, 0.4853, 0.5401, 0.3791, 1.0000, 0.4586, 0.5401,
        0.3791, 0.7848])

We compute the new embeddings as the sum of the noncontextual embeddings weighted by the cosine similarity. We have contextual embeddings.

In [19]:
new_embeddings_ship = (0.35 * embeddings_dict['i'] +
                       0.42 * embeddings_dict['must'] +
                       0.41 * embeddings_dict['go'] +
                       0.49 * embeddings_dict['back'] +
                       0.54 * embeddings_dict['to'] +
                       0.38 * embeddings_dict['my'] +
                       1.00 * embeddings_dict['ship'] +
                       0.46 * embeddings_dict['and'] +
                       0.54 * embeddings_dict['to'] +
                       0.38 * embeddings_dict['my'] +
                       0.78 * embeddings_dict['crew'])
new_embeddings_ship

tensor([  3.2289,   0.6422,   1.4712,  -2.3538,   2.2414,  -0.4237,  -4.1052,
          2.6216,   0.1719,  -2.4324,   1.3882,   3.7241,  -1.9721,   1.1893,
          2.2511,   0.9502,  -0.7646,   1.0289,  -3.0553,  -3.6306,   0.8305,
          2.9299,   1.3221,  -0.7092,   2.9745, -10.5959,  -1.3168,   0.2059,
          3.5457,  -2.7711,  18.2672,   2.4817,  -3.5887,   0.3297,   1.2718,
          0.6539,   1.5873,   0.0195,   0.7724,  -1.4620,  -0.2067,  -1.2464,
          2.1504,  -0.1811,  -0.5026,  -0.2888,  -0.5060,  -1.9676,  -0.0605,
         -0.6725])

Exact computation with torch

In [20]:
(attn_cos_weights(X_o) @ X_o)[6]

tensor([ 3.2319e+00,  6.4082e-01,  1.4718e+00, -2.3434e+00,  2.2358e+00,
        -4.1877e-01, -4.1002e+00,  2.6211e+00,  1.8010e-01, -2.4360e+00,
         1.3923e+00,  3.7188e+00, -1.9603e+00,  1.1980e+00,  2.2394e+00,
         9.3763e-01, -7.7049e-01,  1.0349e+00, -3.0615e+00, -3.6259e+00,
         8.3401e-01,  2.9281e+00,  1.3165e+00, -7.1303e-01,  2.9667e+00,
        -1.0567e+01, -1.3099e+00,  2.0283e-01,  3.5362e+00, -2.7571e+00,
         1.8220e+01,  2.4698e+00, -3.5804e+00,  3.2604e-01,  1.2760e+00,
         6.5701e-01,  1.5889e+00,  1.1571e-02,  7.6620e-01, -1.4560e+00,
        -2.0362e-01, -1.2484e+00,  2.1550e+00, -1.8767e-01, -5.0253e-01,
        -2.9128e-01, -5.1006e-01, -1.9596e+00, -5.8853e-02, -6.7380e-01])

## Self-attention

Vaswani et al. (2017) defined attention as:
$$
\text{Attention}({Q}, {K}, {Q}) = \text{softmax}(\frac{{Q}  {K}^\intercal}{\sqrt{d_k}})  {V},
$$
where
$$
\begin{array}{lcl}
{Q} &=& {X} {W}^Q,   \\
{K} &=& {X} {W}^K , \\
{V} &=& {X} {W}^V.\\
\end{array}
$$
and ${X}$ represents complete input sequence (all the tokens).

$d_k$ is the dimension of the input and $\sqrt{d_k}$ a scaling factor. The $\text{softmax}$ function is defined as:
$$
\text{softmax}(x_1, x_2, ..., x_j, ..., x_n) = (\frac{e^{x_1}}{\sum_{i=1}^n e^{x_i}}, \frac{e^{x_2}}{\sum_{i=1}^n e^{x_i}}, ..., \frac{e^{x_j}}{\sum_{i=1}^n e^{x_i}}, ..., \frac{e^{x_n}}{\sum_{i=1}^n e^{x_i}})
$$

We omit the weight matrices and we use the same embeddings for ${Q}$, ${K}$, and ${Q}$: GloVe embeddings

For the matrix above, self attention, $\text{softmax}(\frac{{Q}  {K}^\intercal}{\sqrt{d_k}})$,  for _ship_ yields:

In [21]:
dk = embeddings_dict['i'].size(dim=0)
dk = torch.tensor(dk)
dk

tensor(50)

In [22]:
attn_weights_o = F.softmax(
    X_o @ X_o.T/torch.sqrt(dk), dim=-1)
attn_weights_o[6]

tensor([0.0303, 0.0302, 0.0276, 0.0407, 0.0459, 0.0343, 0.5530, 0.0297, 0.0459,
        0.0343, 0.1281])

The scaled and normalized attention weights

In [23]:
def print_attn_weights(words, embeddings_dict):
    embeddings = embedding_matrix(words, embeddings_dict)
    sent_length, dk = embeddings.size()
    attn_weights = F.softmax(embeddings @ embeddings.T /
                             torch.sqrt(torch.tensor(dk)), dim=-1)
    print('\t', end='')
    for i in range(sent_length):
        print(words[i], end='\t')
    print()
    for i in range(sent_length):
        print(words[i], end='\t')
        for j in range(sent_length):
            print(f"{attn_weights[i,j]:.2f}", end='\t')
        print()

In [24]:
print_attn_weights(words_o, embeddings_dict)

	i	must	go	back	to	my	ship	and	to	my	crew	
i	0.36	0.05	0.07	0.05	0.04	0.19	0.01	0.02	0.04	0.19	0.01	
must	0.14	0.20	0.10	0.06	0.11	0.10	0.03	0.05	0.11	0.10	0.02	
go	0.18	0.09	0.14	0.09	0.08	0.13	0.02	0.04	0.08	0.13	0.02	
back	0.14	0.05	0.09	0.19	0.08	0.12	0.03	0.06	0.08	0.12	0.03	
to	0.11	0.11	0.09	0.09	0.15	0.08	0.04	0.07	0.15	0.08	0.03	
my	0.19	0.03	0.05	0.04	0.03	0.29	0.01	0.02	0.03	0.29	0.01	
ship	0.03	0.03	0.03	0.04	0.05	0.03	0.55	0.03	0.05	0.03	0.13	
and	0.10	0.08	0.07	0.10	0.12	0.09	0.04	0.15	0.12	0.09	0.04	
to	0.11	0.11	0.09	0.09	0.15	0.08	0.04	0.07	0.15	0.08	0.03	
my	0.19	0.03	0.05	0.04	0.03	0.29	0.01	0.02	0.03	0.29	0.01	
crew	0.06	0.05	0.05	0.06	0.05	0.06	0.21	0.04	0.05	0.06	0.31	


For _ship:_

In [25]:
attn_weights_o[6]

tensor([0.0303, 0.0302, 0.0276, 0.0407, 0.0459, 0.0343, 0.5530, 0.0297, 0.0459,
        0.0343, 0.1281])

We have the weights of 55% for _ship_ and 13% for _crew_, the rest from the other words.

And the new contextual embedding is for _ship_ is a linear combination:

In [26]:
self_attention_ship = (0.03 * embeddings_dict['i'] +
                       0.03 * embeddings_dict['must'] +
                       0.03 * embeddings_dict['go'] +
                       0.04 * embeddings_dict['back'] +
                       0.05 * embeddings_dict['to'] +
                       0.03 * embeddings_dict['my'] +
                       0.55 * embeddings_dict['ship'] +
                       0.03 * embeddings_dict['and'] +
                       0.05 * embeddings_dict['to'] +
                       0.03 * embeddings_dict['my'] +
                       0.13 * embeddings_dict['crew'])
self_attention_ship

tensor([ 1.0442,  0.0966,  0.3467, -0.4238,  0.2203, -0.0956, -0.9915,  0.6637,
         0.4368, -0.7943,  0.5639,  0.9838,  0.0240,  0.5066,  0.0732, -0.1740,
        -0.3322,  0.5614, -1.1613, -0.5717,  0.4356,  0.4120, -0.0659, -0.3336,
         0.6579, -1.7421, -0.0344,  0.1440,  0.8547, -0.1430,  2.6614, -0.0553,
        -0.5376,  0.3057,  0.4068,  0.2231,  0.3959, -0.2940, -0.1163, -0.1340,
         0.1709, -0.5332,  0.9552, -0.4178, -0.1058, -0.1715, -0.2251, -0.3923,
         0.2098, -0.3625])

Exact and complete computation of the whole matrix with torch of 
$$
\text{softmax}(\frac{{Q}  {K}^\intercal}{\sqrt{d_k}})  {V} :
$$

In [27]:
self_attention_output_o = attn_weights_o @ X_o

The contextual embeddings for _ship:_

In [28]:
self_attention_output_o[6]

tensor([ 1.0387,  0.1033,  0.3426, -0.4320,  0.2237, -0.0958, -0.9926,  0.6662,
         0.4424, -0.7942,  0.5638,  0.9921,  0.0205,  0.5082,  0.0743, -0.1773,
        -0.3408,  0.5675, -1.1545, -0.5718,  0.4288,  0.4191, -0.0658, -0.3339,
         0.6682, -1.7473, -0.0485,  0.1531,  0.8642, -0.1447,  2.6571, -0.0545,
        -0.5343,  0.3160,  0.4041,  0.2277,  0.3958, -0.2916, -0.1126, -0.1385,
         0.1744, -0.5375,  0.9499, -0.4145, -0.1039, -0.1755, -0.2213, -0.3995,
         0.2119, -0.3610])

We can now write an `attention` function 

In [29]:
def attention(Q, K, V):
    d_k = K.size(dim=-1)
    attn_weights = F.softmax(Q @ K.T/math.sqrt(d_k), dim=-1)
    attn_output = attn_weights @ V
    return attn_output, attn_weights

The word _ship_ in another context: _We process and ship your order_

In [30]:
attention_output_a, attn_weights_a = attention(X_a, X_a, X_a)

Attention weights for _ship:_

In [31]:
attn_weights_a[3]

tensor([0.0431, 0.0258, 0.0419, 0.7811, 0.0490, 0.0590])

In [32]:
print_attn_weights(words_a, embeddings_dict)

	we	process	and	ship	your	order	
we	0.61	0.06	0.06	0.02	0.20	0.05	
process	0.17	0.50	0.08	0.03	0.11	0.11	
and	0.22	0.12	0.30	0.08	0.15	0.13	
ship	0.04	0.03	0.04	0.78	0.05	0.06	
your	0.14	0.03	0.03	0.02	0.74	0.04	
order	0.16	0.13	0.10	0.09	0.18	0.34	


The new contextual embeddings for _ship:_

In [33]:
attention_output_a[3]

tensor([ 1.2758,  0.1034,  0.2720, -0.4776,  0.1746, -0.1060, -0.9901,  0.6328,
         0.6967, -0.8847,  0.7106,  1.2264,  0.2491,  0.5023, -0.1277, -0.2361,
        -0.3709,  0.6545, -1.2587, -0.5332,  0.6681,  0.1687, -0.2567, -0.4218,
         0.6960, -1.7077, -0.0052,  0.1572,  1.0763,  0.0410,  2.5467, -0.3418,
        -0.5414,  0.4175,  0.4147,  0.2666,  0.3770, -0.4228, -0.2462, -0.0377,
         0.3202, -0.7298,  1.2020, -0.5636, -0.0899, -0.1845, -0.2390, -0.4307,
         0.3828, -0.4905])

## Attention class

In [34]:
class Attention(nn.Module):
    def __init__(self, d_model, d_k):
        super().__init__()
        self.WQ = nn.Linear(d_model, d_k)
        self.WK = nn.Linear(d_model, d_k)
        self.WV = nn.Linear(d_model, d_k)

    def forward(self, X):
        attn_output, attn_weights = attention(self.WQ(X),
                                              self.WK(X),
                                              self.WV(X))
        return attn_output, attn_weights

In [35]:
attn = Attention(X_o.size(dim=1), X_o.size(dim=1))
attn(X_o)

(tensor([[-0.3306, -0.0888, -0.3374,  0.4868,  0.6488, -0.3561, -0.0859,  0.5052,
          -0.0991, -0.6442, -0.5368,  0.2956,  0.9112,  0.0546,  0.0191,  0.2036,
          -0.5131,  0.3797,  0.2747,  0.0123, -0.1460,  0.2397,  0.3909, -0.1393,
           0.1060,  0.0350, -0.3627, -0.0949,  0.0245,  0.4166,  0.1087,  0.0238,
           0.4243, -0.3764,  0.2428, -0.5267,  0.2206,  0.0780,  0.3299, -0.6120,
           0.2306, -0.1049,  0.5755,  0.2655,  0.4114, -0.2689,  0.2682, -0.4302,
           0.4807,  0.0388],
         [-0.3334, -0.0812, -0.3397,  0.4612,  0.6475, -0.3204, -0.0774,  0.4791,
          -0.1121, -0.6310, -0.5556,  0.2988,  0.8794,  0.0519,  0.0175,  0.1988,
          -0.5001,  0.3794,  0.2609,  0.0229, -0.1477,  0.2269,  0.3815, -0.1764,
           0.1138,  0.0248, -0.3662, -0.0695,  0.0180,  0.4271,  0.0552,  0.0331,
           0.4242, -0.3878,  0.2463, -0.4775,  0.2158,  0.0882,  0.3340, -0.6200,
           0.2451, -0.1033,  0.5640,  0.2509,  0.4107, -0.2838,  0.26

## PyTorch implementation
 
PyTorch has an implementation of self-attention encapsulated in the `MultiheadAttention` class. Before going to the attention module, the query, key value, goes through a linear layer. The output also goes through a linear layer. These three layers are initialized with Xavier's algorithm.

In [55]:
from torch.nn import MultiheadAttention

att_layer = MultiheadAttention(50,
                               1,
                               bias=False,
                               batch_first=True)

In [56]:
(attn_output, attn_weights) = att_layer(X_o, X_o, X_o)

The attention weights for _ship:_

In [57]:
attn_weights[6]

tensor([0.1204, 0.1069, 0.0987, 0.0866, 0.0862, 0.1108, 0.0522, 0.0830, 0.0862,
        0.1108, 0.0581], grad_fn=<SelectBackward0>)

In [58]:
attn_output[6]

tensor([ 0.5735,  0.1608,  0.0185,  0.2101, -0.0856,  0.8727,  0.2303, -0.4363,
         0.0050,  0.0262,  0.1571,  0.1330,  0.1806, -0.1861,  0.3188,  0.3523,
         0.4043,  0.6015, -0.0872, -0.3015, -0.6661, -0.3041, -0.6365, -0.6599,
         0.2232,  0.2704, -0.4750, -0.1472, -0.0652,  0.1839,  0.0737, -0.0938,
         0.1181, -0.3653, -0.0229,  0.2583,  0.3538,  0.5438, -0.2840,  0.1109,
        -0.3475, -0.2976,  0.3744,  0.3839, -0.1429, -0.4423,  0.2946,  0.1972,
        -0.2997, -0.6484], grad_fn=<SelectBackward0>)

### The initial dense layers

The weight initial values with the 4 matrices

In [59]:
att_layer.state_dict()

OrderedDict([('in_proj_weight',
              tensor([[ 0.0002, -0.0682, -0.0661,  ..., -0.0809, -0.1216, -0.0408],
                      [-0.0031, -0.0702, -0.0123,  ...,  0.0785, -0.0356, -0.1359],
                      [ 0.0674,  0.0228,  0.1176,  ..., -0.1003, -0.1360,  0.0018],
                      ...,
                      [-0.1508,  0.1119, -0.1293,  ..., -0.0043, -0.0096, -0.0231],
                      [-0.1635, -0.0303, -0.0765,  ..., -0.0585,  0.1543, -0.0684],
                      [ 0.1146,  0.0127, -0.1721,  ...,  0.0625, -0.1184,  0.0530]])),
             ('out_proj.weight',
              tensor([[ 1.2047e-01,  1.3827e-02,  7.9680e-02,  ..., -9.2163e-03,
                       -5.6446e-03,  7.0257e-02],
                      [ 2.7449e-03, -9.1909e-02, -8.6046e-02,  ..., -1.0462e-01,
                        8.8011e-02, -2.3508e-02],
                      [ 1.3978e-01, -9.3539e-02, -4.3567e-02,  ...,  6.9287e-03,
                       -1.1058e-01,  3.8921e-03],
        

The three input matrices are concatenated

In [60]:
att_layer.state_dict()['in_proj_weight'].size()

torch.Size([150, 50])

The output matrix

In [61]:
att_layer.state_dict()['out_proj.weight'].size()

torch.Size([50, 50])

### By-passing the dense layers

We create identity matrices to pass through the dense layers and recover the attention output values and weights

In [62]:
i_50 = torch.eye(50)
i_50

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [63]:
att_layer.state_dict()['out_proj.weight'][:] = i_50

In [64]:
att_layer.state_dict()['in_proj_weight'].size()

torch.Size([150, 50])

In [65]:
i_3_50 = torch.vstack((i_50, i_50, i_50))
i_3_50.size()

torch.Size([150, 50])

In [66]:
att_layer.state_dict()['in_proj_weight'][:] = i_3_50

In [67]:
att_layer.state_dict()

OrderedDict([('in_proj_weight',
              tensor([[1., 0., 0.,  ..., 0., 0., 0.],
                      [0., 1., 0.,  ..., 0., 0., 0.],
                      [0., 0., 1.,  ..., 0., 0., 0.],
                      ...,
                      [0., 0., 0.,  ..., 1., 0., 0.],
                      [0., 0., 0.,  ..., 0., 1., 0.],
                      [0., 0., 0.,  ..., 0., 0., 1.]])),
             ('out_proj.weight',
              tensor([[1., 0., 0.,  ..., 0., 0., 0.],
                      [0., 1., 0.,  ..., 0., 0., 0.],
                      [0., 0., 1.,  ..., 0., 0., 0.],
                      ...,
                      [0., 0., 0.,  ..., 1., 0., 0.],
                      [0., 0., 0.,  ..., 0., 1., 0.],
                      [0., 0., 0.,  ..., 0., 0., 1.]]))])

### Multihead attention without the dense layers

We obtain now the same results as the `self_attention()` function for _ship:_

The attention weights for _ship:_

In [68]:
(attn_output, attn_weights) = att_layer(X_o, X_o, X_o)

The attention vector for _ship:_

In [69]:
attn_weights[6]

tensor([0.0303, 0.0302, 0.0276, 0.0407, 0.0459, 0.0343, 0.5530, 0.0297, 0.0459,
        0.0343, 0.1281], grad_fn=<SelectBackward0>)

The embedding vector for _ship_

In [70]:
attn_output[6]

tensor([ 1.0387,  0.1033,  0.3426, -0.4320,  0.2237, -0.0958, -0.9926,  0.6662,
         0.4424, -0.7942,  0.5638,  0.9921,  0.0205,  0.5082,  0.0743, -0.1773,
        -0.3408,  0.5675, -1.1545, -0.5718,  0.4288,  0.4191, -0.0658, -0.3339,
         0.6682, -1.7473, -0.0485,  0.1531,  0.8642, -0.1447,  2.6571, -0.0545,
        -0.5343,  0.3160,  0.4041,  0.2277,  0.3958, -0.2916, -0.1126, -0.1385,
         0.1744, -0.5375,  0.9499, -0.4145, -0.1039, -0.1755, -0.2213, -0.3995,
         0.2119, -0.3610], grad_fn=<SelectBackward0>)

## Multihead

In [71]:
att_layer_5 = MultiheadAttention(50,
                                 5,
                                 bias=False,
                                 batch_first=True)

In [72]:
attn_output, attn_weights = att_layer_5(X_o, X_o, X_o)

In [73]:
attn_output.size()

torch.Size([11, 50])

In [74]:
attn_weights.size()

torch.Size([11, 11])

In [75]:
att_layer_5.state_dict()

OrderedDict([('in_proj_weight',
              tensor([[-0.0433,  0.1710, -0.1359,  ...,  0.0287, -0.1654,  0.1329],
                      [-0.1624, -0.1484, -0.0658,  ..., -0.1378, -0.0925, -0.0543],
                      [ 0.0664,  0.1332, -0.1427,  ...,  0.1397, -0.0928, -0.1521],
                      ...,
                      [ 0.0910,  0.1242, -0.0305,  ..., -0.1691,  0.1390,  0.0234],
                      [ 0.0617, -0.0719, -0.0593,  ...,  0.1258, -0.0982,  0.1115],
                      [-0.1105,  0.0532,  0.1002,  ...,  0.1498, -0.0320, -0.0927]])),
             ('out_proj.weight',
              tensor([[-0.1033, -0.0316, -0.0938,  ..., -0.0324,  0.0871,  0.0448],
                      [-0.1056, -0.0875,  0.0452,  ..., -0.0211, -0.1196,  0.1062],
                      [ 0.1396,  0.1287, -0.0455,  ...,  0.0557, -0.0172,  0.0109],
                      ...,
                      [ 0.0929, -0.0283, -0.0895,  ...,  0.0422,  0.0151, -0.0953],
                      [-0.0057, -0.014

In [76]:
att_layer_5.state_dict()['in_proj_weight'].size()

torch.Size([150, 50])

In [77]:
att_layer_5.state_dict()['out_proj.weight'].size()

torch.Size([50, 50])

## Test with a simple matrix

Three words, dimension of embeddings: 4

In [59]:
X_test = torch.tensor([[1.0, 0.0, 0.0, 1.0],
                       [0.0, 1.5, 1.0, 1.0],
                       [0.0, 1.0, 1.0, 1.0]])

In [ ]:
X_test.size()

### Self-attention
We use the function above

In [ ]:
attention(X_test, X_test, X_test)

### Multihead attention from PyTorch

In [62]:
att_layer = MultiheadAttention(4,
                               1,
                               bias=False)

The multihead attention uses a Xavier initialization of the dense layers. The results will be different for those of `attention()`

In [ ]:
att_layer(X_test,
          X_test,
          X_test)

Weights of the dense layers

In [ ]:
att_layer.state_dict()

### By-passing the dense layers

We use weights of identity matrices

In [65]:
i_4 = torch.eye(4)

In [66]:
att_layer.state_dict()['out_proj.weight'][:] = i_4

In [ ]:
i_3_4 = torch.vstack((i_4, i_4, i_4))
i_3_4.size()

We set these weights

In [68]:
att_layer.state_dict()['in_proj_weight'][:] = i_3_4

In [ ]:
att_layer.state_dict()

Now we have the same results as with `attention()`

In [ ]:
att_layer(X_test,
          X_test,
          X_test)